### Converting InChi Keys of the compounds on the test dataset into IUPAC names (or to plain English synonyms if available)

Imports:

In [1]:
import pandas as pd
import requests
import time

In [2]:
df = pd.read_csv('../raw_data/tox21_compoundData.csv')

In [3]:
df_ids = df[['ID', 'inchikey', 'set']]

In [4]:
df_ids_test = df_ids[df_ids['set'] == 'test'].reset_index(drop=True)
df_ids_test

,ID,inchikey,set
0,NCGC00261900-01,ABCSSKWSUJMJCP-WQDFMEOSSA-N,test
1,NCGC00260869-01,DMRMZQATXPQOTP-XIIVPSJUSA-M,test
2,NCGC00261776-01,ACVGWSKVRYFWRP-UHFFFAOYSA-N,test
3,NCGC00261380-01,LCHACRBDLUKTTM-UHFFFAOYSA-N,test
4,NCGC00261842-01,MQUQNUAYKLCRME-INIZCTEOSA-N,test
...,...,...,...
642,NCGC00357168-01,SHLSSLVZXJBVHE-UHFFFAOYSA-N,test
643,NCGC00357283-01,HUHGPYXAVBJSJV-UHFFFAOYSA-N,test
644,NCGC00357210-01,XOHZHMUQBFJTNH-UHFFFAOYSA-N,test
645,NCGC00357118-01,MNHVNIJQQRJYDH-UHFFFAOYSA-N,test


### Functions that collect the names of the compounds from PubChem based on their InChi Keys 

In [5]:
def get_smiles(inchikey):
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{inchikey}/property/CanonicalSMILES/JSON'
    try:
        res = requests.get(url, timeout=10)
        if res.status_code == 200:
            return res.json()['PropertyTable']['Properties'][0]['CanonicalSMILES']
    except:
        pass
    return None

In [6]:
def get_iupac_name(inchikey):
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{inchikey}/property/IUPACName/JSON'
    try:
        res = requests.get(url, timeout=10)
        if res.status_code == 200:
            data = res.json()
            return data['PropertyTable']['Properties'][0]['IUPACName']
    except:
        pass
    return None


In [7]:
def get_title(inchikey):
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{inchikey}/property/Title/JSON'
    try:
        res = requests.get(url, timeout=10)
        if res.status_code == 200:
            data = res.json()
            return data['PropertyTable']['Properties'][0]['Title']
    except:
        pass
    return None

In [8]:
smiles_list = []
iupac_names = []
titles = []

total = len(df_ids_test)


In [9]:
for i, inchikey in enumerate(df_ids_test['inchikey'], 1):
    smiles = get_smiles(inchikey)
    iupac = get_iupac_name(inchikey)
    title = get_title(inchikey)

    smiles_list.append(smiles)
    iupac_names.append(iupac)
    titles.append(title)
    
    time.sleep(0.2)  
    if i % 50 == 0:
        print(f"Processed {i} of {total} compounds")

Processed 50 of 647 compounds
Processed 100 of 647 compounds
Processed 150 of 647 compounds
Processed 200 of 647 compounds
Processed 250 of 647 compounds
Processed 300 of 647 compounds
Processed 350 of 647 compounds
Processed 400 of 647 compounds
Processed 450 of 647 compounds
Processed 500 of 647 compounds
Processed 550 of 647 compounds
Processed 600 of 647 compounds


In [10]:
df_ids_test['smiles'] = smiles_list
df_ids_test['iupac_name'] = iupac_names
df_ids_test['title'] = titles


In [11]:
df_ids_test

,ID,inchikey,set,smiles,iupac_name,title
0,NCGC00261900-01,ABCSSKWSUJMJCP-WQDFMEOSSA-N,test,CC12CC=C3C(C1CCC2C(=O)CN4CCN(CC4)C5=NC(=NC(=C5...,"(Z)-but-2-enedioic acid;(8S,10S,13S,14S,17S)-1...",U-74389G maleate
1,NCGC00260869-01,DMRMZQATXPQOTP-XIIVPSJUSA-M,test,C1C2C(C(C(O2)N3C4=NC=NC(=C4N=C3Br)N)O)OP(=O)(O...,sodium;(7aR)-6-(6-amino-8-bromopurin-9-yl)-2-o...,None
2,NCGC00261776-01,ACVGWSKVRYFWRP-UHFFFAOYSA-N,test,C1CN2C(=NC3=CC=CC=C3C2=O)C4=C1C5=CC=CC=C5N4,"3,13,21-triazapentacyclo[11.8.0.02,10.04,9.015...",Rutecarpine
3,NCGC00261380-01,LCHACRBDLUKTTM-UHFFFAOYSA-N,test,C1=CC(=CC=C1C(C2=CC=C(C=C2)F)OCCCC3=CN=CN3)F.Cl,5-[3-[bis(4-fluorophenyl)methoxy]propyl]-1H-im...,3-(1H-Imidazol-4-yl)propyl di(p-fluorophenyl)m...
4,NCGC00261842-01,MQUQNUAYKLCRME-INIZCTEOSA-N,test,CC1=CC=C(C=C1)S(=O)(=O)NC(CC2=CC=CC=C2)C(=O)CCl,N-[(2S)-4-chloro-3-oxo-1-phenylbutan-2-yl]-4-m...,N-Tosyl-L-phenylalanyl chloromethyl ketone
...,...,...,...,...,...,...
642,NCGC00357168-01,SHLSSLVZXJBVHE-UHFFFAOYSA-N,test,C(CO)CS,3-sulfanylpropan-1-ol,3-Mercapto-1-propanol
643,NCGC00357283-01,HUHGPYXAVBJSJV-UHFFFAOYSA-N,test,C1N(CN(CN1CCO)CCO)CCO,"2-[3,5-bis(2-hydroxyethyl)-1,3,5-triazinan-1-y...",Triazinetriethanol
644,NCGC00357210-01,XOHZHMUQBFJTNH-UHFFFAOYSA-N,test,CN1C(=NN=N1)S,1-methyltetrazole-5-thiol,None
645,NCGC00357118-01,MNHVNIJQQRJYDH-UHFFFAOYSA-N,test,C1CC1(C(CC2=CC=CC=C2Cl)(CN3C(=S)N=CN3)O)Cl,2-[2-(1-chlorocyclopropyl)-3-(2-chlorophenyl)-...,Prothioconazole


In [12]:
df_ids_test['iupac_name'][1]

'sodium;(7aR)-6-(6-amino-8-bromopurin-9-yl)-2-oxido-2-oxo-4a,6,7,7a-tetrahydro-4H-furo[3,2-d][1,3,2]dioxaphosphinin-7-ol'

In [13]:
path_features_test = "../raw_data/tox21_dense_test.csv.gz" 
X_test = pd.read_csv(path_features_test)
X_test.shape

(647, 802)

In [14]:
X_test.head()

,Unnamed: 0,AW,AWeight,Arto,BertzCT,Chi0,Chi1,Chi10,Chi2,Chi3,...,W3D,W3DH,WNSA1,WNSA2,WNSA3,WPSA1,WPSA2,WPSA3,grav,rygr
0,NCGC00261900-01,2.612482e+07,12.688,2.226,3.226,37.329,25.440,3.663,24.200,20.222,...,9687.312,42351.907,194.444,-2518.829,-83.110,772.051,10001.075,131.633,145.967,5.499
1,NCGC00260869-01,8.333337e+06,17.500,2.167,2.923,16.353,10.872,1.193,11.116,9.279,...,1256.410,2621.885,104.011,-475.829,-33.456,219.411,1003.763,76.703,76.043,3.728
2,NCGC00261776-01,4.074000e+00,12.464,2.364,3.043,14.681,10.826,2.149,9.980,9.469,...,1072.430,3152.648,93.486,-341.628,-21.327,174.791,638.757,32.885,45.933,3.657
3,NCGC00261380-01,8.000005e+06,13.827,2.080,2.845,16.778,11.720,0.777,10.139,8.207,...,1408.177,4596.402,127.215,-519.799,-27.729,199.061,813.323,35.712,58.214,3.659
4,NCGC00261842-01,4.838000e+00,14.509,2.087,2.880,16.872,10.920,0.413,10.035,7.719,...,1217.075,4343.460,134.802,-816.522,-55.496,192.858,1168.142,33.190,57.065,3.635


In [15]:
X_test = X_test.rename(columns={'Unnamed: 0': 'ID'})

In [16]:
X_test_with_names = df_ids_test.merge(X_test, how="inner", on="ID")

### Dataframe for prediction in the backend

In [17]:
X_test_with_names

,ID,inchikey,set,smiles,iupac_name,title,AW,AWeight,Arto,BertzCT,...,W3D,W3DH,WNSA1,WNSA2,WNSA3,WPSA1,WPSA2,WPSA3,grav,rygr
0,NCGC00261900-01,ABCSSKWSUJMJCP-WQDFMEOSSA-N,test,CC12CC=C3C(C1CCC2C(=O)CN4CCN(CC4)C5=NC(=NC(=C5...,"(Z)-but-2-enedioic acid;(8S,10S,13S,14S,17S)-1...",U-74389G maleate,2.612482e+07,12.688,2.226,3.226,...,9687.312,42351.907,194.444,-2518.829,-83.110,772.051,10001.075,131.633,145.967,5.499
1,NCGC00260869-01,DMRMZQATXPQOTP-XIIVPSJUSA-M,test,C1C2C(C(C(O2)N3C4=NC=NC(=C4N=C3Br)N)O)OP(=O)(O...,sodium;(7aR)-6-(6-amino-8-bromopurin-9-yl)-2-o...,None,8.333337e+06,17.500,2.167,2.923,...,1256.410,2621.885,104.011,-475.829,-33.456,219.411,1003.763,76.703,76.043,3.728
2,NCGC00261776-01,ACVGWSKVRYFWRP-UHFFFAOYSA-N,test,C1CN2C(=NC3=CC=CC=C3C2=O)C4=C1C5=CC=CC=C5N4,"3,13,21-triazapentacyclo[11.8.0.02,10.04,9.015...",Rutecarpine,4.074000e+00,12.464,2.364,3.043,...,1072.430,3152.648,93.486,-341.628,-21.327,174.791,638.757,32.885,45.933,3.657
3,NCGC00261380-01,LCHACRBDLUKTTM-UHFFFAOYSA-N,test,C1=CC(=CC=C1C(C2=CC=C(C=C2)F)OCCCC3=CN=CN3)F.Cl,5-[3-[bis(4-fluorophenyl)methoxy]propyl]-1H-im...,3-(1H-Imidazol-4-yl)propyl di(p-fluorophenyl)m...,8.000005e+06,13.827,2.080,2.845,...,1408.177,4596.402,127.215,-519.799,-27.729,199.061,813.323,35.712,58.214,3.659
4,NCGC00261842-01,MQUQNUAYKLCRME-INIZCTEOSA-N,test,CC1=CC=C(C=C1)S(=O)(=O)NC(CC2=CC=CC=C2)C(=O)CCl,N-[(2S)-4-chloro-3-oxo-1-phenylbutan-2-yl]-4-m...,N-Tosyl-L-phenylalanyl chloromethyl ketone,4.838000e+00,14.509,2.087,2.880,...,1217.075,4343.460,134.802,-816.522,-55.496,192.858,1168.142,33.190,57.065,3.635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,NCGC00357168-01,SHLSSLVZXJBVHE-UHFFFAOYSA-N,test,C(CO)CS,3-sulfanylpropan-1-ol,3-Mercapto-1-propanol,2.000000e+00,16.820,1.600,1.157,...,25.161,207.065,25.378,-34.751,-8.135,45.801,62.715,7.705,6.893,1.868
643,NCGC00357283-01,HUHGPYXAVBJSJV-UHFFFAOYSA-N,test,C1N(CN(CN1CCO)CCO)CCO,"2-[3,5-bis(2-hydroxyethyl)-1,3,5-triazinan-1-y...",Triazinetriethanol,3.714000e+00,13.208,2.000,2.134,...,406.748,2632.001,51.600,-186.503,-25.487,151.500,547.566,24.878,29.311,2.976
644,NCGC00357210-01,XOHZHMUQBFJTNH-UHFFFAOYSA-N,test,CN1C(=NN=N1)S,1-methyltetrazole-5-thiol,None,1.905000e+00,16.017,2.000,2.295,...,49.621,146.562,34.623,-39.767,-6.189,38.729,44.481,7.121,12.809,1.779
645,NCGC00357118-01,MNHVNIJQQRJYDH-UHFFFAOYSA-N,test,C1CC1(C(CC2=CC=CC=C2Cl)(CN3C(=S)N=CN3)O)Cl,2-[2-(1-chlorocyclopropyl)-3-(2-chlorophenyl)-...,Prothioconazole,4.186000e+00,15.674,2.190,2.851,...,902.831,2855.278,112.538,-443.321,-28.525,142.805,562.539,28.106,55.845,3.282


### Dictionary of names for the frontend

In [18]:
compound_dict = X_test_with_names.set_index('ID')[['smiles', 'iupac_name', 'title']].to_dict(orient='index')

In [19]:
compound_dict

{'NCGC00261900-01': {'smiles': 'CC12CC=C3C(C1CCC2C(=O)CN4CCN(CC4)C5=NC(=NC(=C5)N6CCCC6)N7CCCC7)CCC8=CC(=O)C=CC83C.C(=CC(=O)O)C(=O)O',
  'iupac_name': '(Z)-but-2-enedioic acid;(8S,10S,13S,14S,17S)-17-[2-[4-(2,6-dipyrrolidin-1-ylpyrimidin-4-yl)piperazin-1-yl]acetyl]-10,13-dimethyl-6,7,8,12,14,15,16,17-octahydrocyclopenta[a]phenanthren-3-one',
  'title': 'U-74389G maleate'},
 'NCGC00260869-01': {'smiles': 'C1C2C(C(C(O2)N3C4=NC=NC(=C4N=C3Br)N)O)OP(=O)(O1)[O-].[Na+]',
  'iupac_name': 'sodium;(7aR)-6-(6-amino-8-bromopurin-9-yl)-2-oxido-2-oxo-4a,6,7,7a-tetrahydro-4H-furo[3,2-d][1,3,2]dioxaphosphinin-7-ol',
  'title': None},
 'NCGC00261776-01': {'smiles': 'C1CN2C(=NC3=CC=CC=C3C2=O)C4=C1C5=CC=CC=C5N4',
  'iupac_name': '3,13,21-triazapentacyclo[11.8.0.02,10.04,9.015,20]henicosa-1(21),2(10),4,6,8,15,17,19-octaen-14-one',
  'title': 'Rutecarpine'},
 'NCGC00261380-01': {'smiles': 'C1=CC(=CC=C1C(C2=CC=C(C=C2)F)OCCCC3=CN=CN3)F.Cl',
  'iupac_name': '5-[3-[bis(4-fluorophenyl)methoxy]propyl]-1H-imidazol

In [21]:
df_compounds = pd.DataFrame.from_dict(compound_dict, orient='index').reset_index()
df_compounds.columns = ['ID', 'smiles', 'iupac_name', 'title']

In [22]:
df_compounds.head()

,ID,smiles,iupac_name,title
0,NCGC00261900-01,CC12CC=C3C(C1CCC2C(=O)CN4CCN(CC4)C5=NC(=NC(=C5...,"(Z)-but-2-enedioic acid;(8S,10S,13S,14S,17S)-1...",U-74389G maleate
1,NCGC00260869-01,C1C2C(C(C(O2)N3C4=NC=NC(=C4N=C3Br)N)O)OP(=O)(O...,sodium;(7aR)-6-(6-amino-8-bromopurin-9-yl)-2-o...,None
2,NCGC00261776-01,C1CN2C(=NC3=CC=CC=C3C2=O)C4=C1C5=CC=CC=C5N4,"3,13,21-triazapentacyclo[11.8.0.02,10.04,9.015...",Rutecarpine
3,NCGC00261380-01,C1=CC(=CC=C1C(C2=CC=C(C=C2)F)OCCCC3=CN=CN3)F.Cl,5-[3-[bis(4-fluorophenyl)methoxy]propyl]-1H-im...,3-(1H-Imidazol-4-yl)propyl di(p-fluorophenyl)m...
4,NCGC00261842-01,CC1=CC=C(C=C1)S(=O)(=O)NC(CC2=CC=CC=C2)C(=O)CCl,N-[(2S)-4-chloro-3-oxo-1-phenylbutan-2-yl]-4-m...,N-Tosyl-L-phenylalanyl chloromethyl ketone


In [23]:
df_compounds.to_csv("compounds.csv", index=False)

In [24]:
df_compounds.to_pickle("compounds.pkl")